In [ ]:
pip install pandas scikit-surprise nltk


1. Define the Custom Dataset:

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


# Create a small custom dataset

In [ ]:
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5],
    'recipe_id': [101, 102, 103, 101, 104, 102, 104, 105, 103, 105, 101],
    'rating': [5, 4, 3, 5, 4, 2, 5, 4, 3, 5, 4]
}

recipes = {
    'recipe_id': [101, 102, 103, 104, 105],
    'name': ['Chicken Biryani Kit', 'Paneer Biryani Kit', 'Mushroom Biryani Kit', 'Salad Kit', 'Roll Kit']
}

# Convert to DataFrames

In [ ]:
interactions_df = pd.DataFrame(data)
recipes_df = pd.DataFrame(recipes)

# Display the datasets

In [ ]:
print("User-Recipe Interactions:")
print(interactions_df)
print("\nRecipes:")
print(recipes_df)



User-Recipe Interactions:
    user_id  recipe_id  rating
0         1        101       5
1         1        102       4
2         1        103       3
3         2        101       5
4         2        104       4
5         3        102       2
6         3        104       5
7         3        105       4
8         4        103       3
9         4        105       5
10        5        101       4

Recipes:
   recipe_id                  name
0        101   Chicken Biryani Kit
1        102    Paneer Biryani Kit
2        103  Mushroom Biryani Kit
3        104             Salad Kit
4        105              Roll Kit


Prepare Data for Collaborative Filtering:

In [ ]:
# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(interactions_df[['user_id', 'recipe_id', 'rating']], reader)


Split Data and Train the Model with Hyperparameter Tuning:

In [ ]:
trainset, testset = train_test_split(surprise_data, test_size=0.2)


# Use Singular Value Decomposition (SVD) algorithm

In [ ]:
algo = SVD()

# Train the algorithm on the trainset

In [ ]:
algo.fit(trainset)

# Predict ratings for the testset

In [ ]:
predictions = algo.test(testset)

# Compute and print the RMSE (Root Mean Squared Error)

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.8547


0.8546518698273022


# Define a function to get recommendations for a user

In [ ]:
def get_recommendations(user_id, algo, recipes_df, n=10):
    # Get the list of all item IDs in the training set
    trainset_items = trainset.all_items()
    all_item_ids = [trainset.to_raw_iid(i) for i in trainset_items]

    # Get a list of item_ids the user has already rated
    user_rated_items = trainset.ur[trainset.to_inner_uid(user_id)]
    user_rated_item_ids = [trainset.to_raw_iid(iid) for (iid, _) in user_rated_items]

    # Filter out items the user has already rated
    items_to_predict = [iid for iid in all_item_ids if iid not in user_rated_item_ids]

    # Predict ratings for the items the user has not rated
    predictions = [algo.predict(user_id, iid) for iid in items_to_predict]

    # Sort the predictions by estimated rating in descending order
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top n recommendations
    top_n = predictions[:n]

    # Return the top n recommended item IDs and estimated ratings
    return [(pred.iid, pred.est) for pred in top_n]

# Example: Get recommendations for a user with user_id 1

In [ ]:
user_id = 1
recommended_items = get_recommendations(user_id, algo, recipes_df, n=3)
print(f"Top 3 recommendations for user {user_id}:")
for i, (item_id, rating) in enumerate(recommended_items, 1):
    recipe_name = recipes_df[recipes_df['recipe_id'] == item_id]['name'].values[0]
    print(f"{i}. Recipe: {recipe_name}, Estimated Rating: {rating:.2f}")

Top 3 recommendations for user 1:
1. Recipe: Salad Kit, Estimated Rating: 4.26
2. Recipe: Roll Kit, Estimated Rating: 4.08
3. Recipe: Mushroom Biryani Kit, Estimated Rating: 3.92
